In [ ]:
!pip install orjson

In [27]:
!pip install PyMuPDF
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 7.3 MB/s eta 0:00:00:00:0100:01


In [7]:
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 439.6 kB/s eta 0:00:00 0:00:01m


In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
from functools import partial
import random
from typing import Dict, Tuple, Callable
from PIL import Image, UnidentifiedImageError
from typing import List, Optional
import pypdf
import orjson
import jsonlines     
import fitz  # PyMuPDF


In [2]:
json_path = Path('/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl')

# Open the JSONL file for reading
with jsonlines.open(json_path) as reader:
    for line_number, line in enumerate(reader, start=1):
        # Process each line as needed
        print(line['pdf'])

print(line_number)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl'

In [ ]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.
"""
# Implements image augmentation

import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD


def alb_wrapper(transform):
    def f(im):
        return transform(image=np.asarray(im))["image"]

    return f


class Erosion(alb.ImageOnlyTransform):
    """
    Apply erosion operation to an image.

    Erosion is a morphological operation that shrinks the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the erosion kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the erosion kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.erode(img, kernel, iterations=1)
        return img


class Dilation(alb.ImageOnlyTransform):
    """
    Apply dilation operation to an image.

    Dilation is a morphological operation that expands the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the dilation kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the dilation kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.dilate(img, kernel, iterations=1)
        return img


class Bitmap(alb.ImageOnlyTransform):
    """
    Apply a bitmap-style transformation to an image.

    This transformation replaces all pixel values below a certain threshold with a specified value.

    Args:
        value (int, optional): The value to replace pixels below the threshold with. Default is 0.
        lower (int, optional): The threshold value below which pixels will be replaced. Default is 200.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, value=0, lower=200, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        self.lower = lower
        self.value = value

    def apply(self, img, **params):
        img = img.copy()
        img[img < self.lower] = self.value
        return img


train_transform = alb_wrapper(
    alb.Compose(
        [
            Bitmap(p=0.05),
            alb.OneOf([Erosion((2, 3)), Dilation((2, 3))], p=0.02),
            alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
            alb.ShiftScaleRotate(
                shift_limit_x=(0, 0.04),
                shift_limit_y=(0, 0.03),
                scale_limit=(-0.15, 0.03),
                rotate_limit=2,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.03,
            ),
            alb.GridDistortion(
                distort_limit=0.05,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.04,
            ),
            alb.Compose(
                [
                    alb.Affine(
                        translate_px=(0, 5), always_apply=True, cval=(255, 255, 255)
                    ),
                    alb.ElasticTransform(
                        p=1,
                        alpha=50,
                        sigma=120 * 0.1,
                        alpha_affine=120 * 0.01,
                        border_mode=0,
                        value=(255, 255, 255),
                    ),
                ],
                p=0.04,
            ),
            alb.RandomBrightnessContrast(0.1, 0.1, True, p=0.03),
            alb.ImageCompression(95, p=0.07),
            alb.GaussNoise(20, p=0.08),
            alb.GaussianBlur((3, 3), p=0.03),
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)
test_transform = alb_wrapper(
    alb.Compose(
        [
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)


In [ ]:
# import albumentations as alb
# from albumentations.pytorch import ToTensorV2

# def alb_wrapper(transform):
#     def f(im):
#         return transform(image=np.asarray(im))["image"]

#     return f

# train_transform = alb_wrapper(
#         alb.Compose(
#         [
#             alb.Compose(
#                 [alb.ShiftScaleRotate(shift_limit=0, scale_limit=(-.15, 0), rotate_limit=1, border_mode=0, interpolation=3,
#                                     value=[255, 255, 255], p=1),
#                 alb.GridDistortion(distort_limit=0.1, border_mode=0, interpolation=3, value=[255, 255, 255], p=.5)], p=.15),
#             # alb.InvertImg(p=.15),
#             alb.RGBShift(r_shift_limit=15, g_shift_limit=15,
#                         b_shift_limit=15, p=0.3),
#             alb.GaussNoise(10, p=.2),
#             alb.RandomBrightnessContrast(.05, (-.2, 0), True, p=0.2),
#             alb.ImageCompression(95, p=.3),
#             alb.ToGray(always_apply=True),
#             alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
#             # alb.Sharpen()
#             ToTensorV2(),
            
#         ]
#     )
# )
# test_transform = alb.Compose(
#     [
#         alb.ToGray(always_apply=True),
#         alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
#         # alb.Sharpen()
#         ToTensorV2(),
#     ]
# )

In [ ]:

import numpy as np
from PIL import Image
import cv2
from torchvision import transforms


from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer
from torchvision.transforms.functional import resize, rotate

class enc():

    def __init__(self):
        self.input_size = [896,16800]
        self.align_long_axis = True
        self.training = True

    def crop_margin(self, img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

    # def to_tensor(self, img: Image.Image):
    #     if self.training:
    #         return train_transform
    #     else:
    #         return test_transform
    def to_tensor(self, img: Image.Image):
        if self.training:
            # return train_transform(image = img)['image']
            return transforms.ToTensor()(img)
        else:
            return transforms.ToTensor()(img)
    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        img_list = []
        # Convert PDF to images using PyMuPDF
        input_size = [896,672]
        doc = fitz.open(pdf_path)
        for page_number in range(len(doc)):
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)

            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (input_size[0] > input_size[1] and img.width > img.height)
                or (input_size[0] < input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(input_size))
            img.thumbnail((input_size[1], input_size[0]))
            delta_width = input_size[1] - img.width
            delta_height = input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            padded_img = ImageOps.expand(img, padding)
            img_list.append(padded_img)

            # page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            # page_tensor = self.to_tensor(img)
            # print(page_tensor)
            # page_tensor=page_tensor.to('cuda')
            # print(page_tensor.shape)
            # if self.input_tensor is None:
            #    self.input_tensor = page_tensor
            # else:
            #     self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=2)


                # Calculate the padding to be added
        target_shape = (3, 896, self.input_size[1])
        original_shape = self.input_tensor.shape
        padding = [0, target_shape[2] - original_shape[2]]

        # Apply padding using torch.nn.functional.pad
        padded_tensor = torch.nn.functional.pad(self.input_tensor, padding)

        return padded_tensor

/home/patidarritesh/miniconda3/envs/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import json
class pdfDataset(Dataset):

    def __init__(self, dataset_path, split: str = "train"):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.pdf_path = []
        self.latex_path = []    

        with jsonlines.open(self.dataset_path) as reader:
            for line_number, line in enumerate(reader, start=1):
                self.pdf_path.append(line['pdf'])
                self.latex_path.append(line['latex'])
        self.dataset_length = line_number

    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx):

        pdf_path = self.pdf_path[idx]
        latex_path = self.latex_path[idx]

        # print("pdf: ", pdf_path)
        # print("latex: ", latex_path)
        encoder = enc()
        input_tensor = encoder.prepare_input(pdf_path, random_padding=True)

        print("input_tensor: ", input_tensor)
        print("input_tensor.shape: ", input_tensor.shape)

        # with open(latex_path, "rb") as f:
        #     gnd_truth_data = f.read()
        #     try:
        #         gnd_truth_data = gnd_truth_data.decode("utf-8")  # Try decoding with UTF-8
        #     except:
        #         gnd_truth_data = gnd_truth_data.decode("latin-1", errors="ignore")  # Fallback to Latin-1, ignore errors

        # # print("gnd_truth_data: ", gnd_truth_data)
        # tokenizer_out = self.nougat_model.decoder.tokenizer(
        #     gnd_truth_data,
        #     max_length=self.max_length,
        #     padding="max_length",
        #     return_token_type_ids=False,
        #     truncation=True,
        #     return_tensors="pt",
        # )
        # input_ids = tokenizer_out["input_ids"].squeeze(0)
        # attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        """      
        # randomly perturb ground truth tokens
        if self.split == "train" and self.perturb:
            # check if we perturb tokens
            unpadded_length = attention_mask.sum()
            while random.random() < 0.1:
                try:
                    pos = random.randint(1, unpadded_length - 2)
                    token = random.randint(
                        23, len(self.nougat_model.decoder.tokenizer) - 1
                    )
                    input_ids[pos] = token
                except ValueError:
                    break"""
        return input_tensor



        # Save input_tensor to JSON file
        # json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
        # with open(json_path, "w") as file:
        #     json.dump(input_tensor.tolist(), file)
        


        return pdf_path, latex_path

In [ ]:
dataset = pdfDataset(dataset_path="/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl")

In [ ]:
for i, sample in enumerate(dataset):
    print("iteration::",i, sample)
    break



input_tensor:  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
input_tensor.shape:  torch.Size([3, 896, 16800])
iteration:: 0 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,

In [3]:
## Padding Details
import json
json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
with open(json_path, "r") as file:
    loaded_data = json.load(file)



for j in range(len(loaded_data[0])):
    count =0
    for i in range(len(loaded_data[0][0])):
        if loaded_data[0][j][i]!=0 and loaded_data[0][0][i]!=1:
            # print(loaded_data[0][0][i])
            count+=1
    print("count: ", count)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'

In [4]:

"""class pdfDataset(Dataset):

    def __init__(self, 
                dataset_path,
                split: str = "traisn",
                root_name: str = "pdf",):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.root_name = root_name
        self.path_to_root = Path(dataset_path).parent
        self.root_name = root_name
        # self.path_to_root = self.path_to_root.parent

        # if not split in self.dataset_path:
        #     pti = self.path_to_root / (template % split + ".jsonl")
        #     if pti.exists():
        #         self.dataset_path = pti
        #     else:
        #         raise ValueError(f'Dataset file for split "{split}" not found: {pti}')
        
        self.dataset_file = None  # mulitprocessing
        
        if self.dataset_file is None:
            self.dataset_file = Path(self.dataset_path).open()
        line = self.dataset_file.readline()
        data: Dict = orjson.loads(line)

        pdf_path: Path = self.path_to_root / self.root_name / data.pop("pdf")
        latex_path: Path = self.path_to_root / self.root_name / data.pop("latex")

        self.dataset = {"pdf": pdf_path, "ground_truth": latex_path}
        self.dataset_length = len(self.dataset)


    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx: int):

        sample = self.dataset[idx]
        # print("idx:", idx)
        # print("sample:", sample)
        return sample
        # if sample is None:
        #     # if sample is broken choose another randomly
        #     return self[random.randint(0, self.dataset_length - 1)]
        # if sample is None or sample["image"] is None or prod(sample["image"].size) == 0:
        #     input_tensor = None
        # else:
        #     input_tensor = self.nougat_model.encoder.prepare_input(
        #         sample["image"], random_padding=self.split == "train"
                
        #     )

        # tokenizer_out = self.nougat_model.decoder.tokenizer(
        #     sample["ground_truth"],
        #     max_length=self.max_length,
        #     padding="max_length",
        #     return_token_type_ids=False,
        #     truncation=True,
        #     return_tensors="pt",
        # )
        # input_ids = tokenizer_out["input_ids"].squeeze(0)
        # attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        # # randomly perturb ground truth tokens
        # if self.split == "train" and self.perturb:
        #     # check if we perturb tokens
        #     unpadded_length = attention_mask.sum()
        #     while random.random() < 0.1:
        #         try:
        #             pos = random.randint(1, unpadded_length - 2)
        #             token = random.randint(
        #                 23, len(self.nougat_model.decoder.tokenizer) - 1
        #             )
        #             input_ids[pos] = token
        #         except ValueError:
        #             break
        # return input_tensor, input_ids, attention_mask
    def __iter__(self):
        for i in range(self.dataset_length):
            yield self[i]"""



'class pdfDataset(Dataset):\n\n    def __init__(self, \n                dataset_path,\n                split: str = "traisn",\n                root_name: str = "pdf",):\n        super().__init__()\n        self.dataset_path = dataset_path\n        self.split = split\n        self.root_name = root_name\n        self.path_to_root = Path(dataset_path).parent\n        self.root_name = root_name\n        # self.path_to_root = self.path_to_root.parent\n\n        # if not split in self.dataset_path:\n        #     pti = self.path_to_root / (template % split + ".jsonl")\n        #     if pti.exists():\n        #         self.dataset_path = pti\n        #     else:\n        #         raise ValueError(f\'Dataset file for split "{split}" not found: {pti}\')\n        \n        self.dataset_file = None  # mulitprocessing\n        \n        if self.dataset_file is None:\n            self.dataset_file = Path(self.dataset_path).open()\n        line = self.dataset_file.readline()\n        data: Dict = 

In [5]:
import fitz  # PyMuPDF
from PIL import Image



def pdf_to_imgs(pdf_path):  
    doc = fitz.open(pdf_path)
    images = []
    for page_number in range(len(doc)):
        page = doc[page_number]
        image = page.get_pixmap()
        # Convert Pixmap to PIL Image
        img = Image.frombytes("RGB", [image.width, image.height], image.samples)
        images.append(img)
    return images



def load_latex_code(self, pdf_path):
    # Implement this function to extract LaTeX code from the PDF
    # You may use a PDF parsing library or any other method
    # Return the extracted LaTeX code as a string
    latex_code = "Your LaTeX code here"
    return latex_code

def images_to_tensor(self, images):
    # Convert a list of images to a PyTorch tensor
    tensor_list = [self.transform(image) for image in images]
    return torch.stack(tensor_list, dim=0)

imgs = pdf_to_imgs("/mnt/NAS/patidarritesh/swin_transformer.pdf")




In [6]:
import numpy as np
import cv2

def crop_margin(img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

In [7]:
image = imgs[0]

image = crop_margin(image.convert("RGB"))

# if ((input_size[0] > input_size[1] and img.width > img.height)
#                 or (input_size[0] < input_size[1] and img.width < img.height)
#             ):
#                 img = rotate(img, angle=-90, expand=True)

In [8]:
imgs[0].size

(612, 792)

In [9]:
image.size


(528, 606)

In [10]:
print("Height: ",image.height)
print("Width: ", image.width)

Height:  606
Width:  528


In [11]:
image.save("output.png")



In [12]:
imgs[0].save("output_1.png")

In [13]:
from torchvision.transforms.functional import resize, rotate
image = resize(image, min([896,224]))


/home/patidarritesh/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
image.size

(224, 257)

In [15]:
x=606*224/528
x

257.09090909090907

In [17]:
def to_tensor(self, img: Image.Image):
        transform_list = []
        
        # Add Resize transformation if needed
        if self.training:
            transform_list.append(transforms.Resize((896, 224)))

        # Add ToTensor transformation
        transform_list.append(transforms.ToTensor())

        composed_transform = transforms.Compose(transform_list)
        return composed_transform(img)
    
    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        input_size= [896,672]
        # Convert PDF to images using PyMuPDF
        doc = fitz.open(pdf_path)
        i=0
        for page_number in range(len(doc)):
            if i==5:
                break
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)
            print("raw img: ",img.size)
            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
                print("img after cropt margin: ",img.size)
                print("-----------------------------------------------")
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (input_size[0] > input_size[1] and img.width > img.height)
                or (input_size[0] < input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(self.input_size))
            img.thumbnail((input_size[1], input_size[0]))
            delta_width = input_size[1] - img.width
            delta_height = input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            
            page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            print("Final transformed page_tensor for single page: ",page_tensor.shape)
            if self.input_tensor is None:
               self.input_tensor = page_tensor
            else:
                self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=2)
            i+=1

        target_shape = (3, self.input_size[0], self.input_size[1])
        original_shape = self.input_tensor.shape
        padding = [0, target_shape[2] - original_shape[2]]

        # Apply padding using torch.nn.functional.pad
        self.input_tensor = torch.nn.functional.pad(self.input_tensor, padding)
        print("input_tensor  after paddig: ",self.input_tensor.shape)
        return self.input_tensor

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 14)

In [18]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer

In [19]:
model = SwinTransformer(
            # img_size=self.input_size,
            # depths=self.encoder_layer,
            # window_size=self.window_size,
            # patch_size=self.patch_size,
            # embed_dim=self.embed_dim,
            # num_heads=self.num_heads,
            # num_classes=0,
        )

/home/patidarritesh/miniconda3/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [20]:
swin_state_dict = timm.create_model(
"swin_base_patch4_window12_384", pretrained=True
).state_dict()
new_swin_state_dict =model.state_dict()

In [21]:
x=swin_state_dict["layers.0.blocks.0.attn.relative_position_bias_table"]

In [22]:
pos_bias = x.unsqueeze(0)[0]
pos_bias.shape

torch.Size([529, 4])

In [23]:
import math
window_size = 7

In [24]:
old_len = int(math.sqrt(len(pos_bias)))
new_len = int(2 * window_size - 1)

In [25]:
print("old_len: ",old_len)
print("new_len: ",new_len)

old_len:  23
new_len:  13


In [26]:
  pos_bias = pos_bias.reshape(1, old_len, old_len, -1).permute(
                        0, 3, 1, 2
                    )

In [27]:
pos_bias.shape

torch.Size([1, 4, 23, 23])

In [28]:
  pos_bias = F.interpolate(
                        pos_bias,
                        size=(new_len, new_len),
                        mode="bicubic",
                        align_corners=False,
                    )

In [29]:
pos_bias.shape

torch.Size([1, 4, 13, 13])

In [30]:
   new_swin_state_dict[x] = (
                        pos_bias.permute(0, 2, 3, 1)
                        .reshape(1, new_len**2, -1)
                        .squeeze(0)
                    )

In [31]:
 new_swin_state_dict[x].shape

torch.Size([169, 4])

In [32]:
# opena a .tex file as text string
file = open("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", "r")
text = file.read()
file.close()

# print text
(text)

'\\importpackages{}\n\\graphicspath{ {./images/} }\n\n\n\\title{Topological surface states in paramagnetic and antiferromagnetic\niron pnictides}\n\\author{Alexander Lau}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\author{Carsten Timm}\n\\email{carsten.timm@tu-dresden.de}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\date{\\today}\n\\begin{abstract}\nThe electronic structure of iron pnictides is topologically nontrivial, leading\nto the appearance of Dirac cones in the band structure for the antiferromagnetic\nphase. Motivated by the analogy with Dirac cones in graphene, we explore the\npossible existence\nof topologically protected surface states. Surprisingly, bands of surface states\nexist even in the paramagnetic state. A realistic five-orbital model\npredicts two such bands. In the antiferromagnetic phase, these surface bands\nsurvive but split. We 

In [33]:
latex_path = "/mnt/NAS/patidarritesh/latex_files/1305.1770.tex"

with open(latex_path, "rb") as f:
    gnd_truth_data = f.read()
    try:
        gnd_truth_data = gnd_truth_data.decode("utf-8")  # Try decoding with UTF-8
    except:
        gnd_truth_data = gnd_truth_data.decode("latin-1", errors="ignore")  # Fallback to Latin-1, ignore errors
print("gnd_truth_data: ", gnd_truth_data)

gnd_truth_data:  \importpackages{}
\graphicspath{ {./images/} }


\title{Topological surface states in paramagnetic and antiferromagnetic
iron pnictides}
\author{Alexander Lau}
\affiliation{Institute of Theoretical Physics, Technische Universit\"at
Dresden, 01062 Dresden, Germany}
\author{Carsten Timm}
\email{carsten.timm@tu-dresden.de}
\affiliation{Institute of Theoretical Physics, Technische Universit\"at
Dresden, 01062 Dresden, Germany}
\date{\today}
\begin{abstract}
The electronic structure of iron pnictides is topologically nontrivial, leading
to the appearance of Dirac cones in the band structure for the antiferromagnetic
phase. Motivated by the analogy with Dirac cones in graphene, we explore the
possible existence
of topologically protected surface states. Surprisingly, bands of surface states
exist even in the paramagnetic state. A realistic five-orbital model
predicts two such bands. In the antiferromagnetic phase, these surface bands
survive but split. We obtain the bulk and

In [34]:
from tokenizers import Tokenizer
from pathlib import Path
from tokenizers import Tokenizer
import orjson
import os

def tokenize_latex_file(file_path, tokenizer_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        return

    # Read the LaTeX file
    try:
        with open(file_path, 'r', encoding='utf-8') as latex_file:
            latex_content = latex_file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as latex_file:
            latex_content = latex_file.read()

    # Initialize the tokenizer
    tokenizer = Tokenizer.from_file(tokenizer_path)

    # Tokenize the LaTeX content
    tokens = tokenizer.encode(latex_content).ids
    print(tokens)
    return tokens
    # # Print the tokens
    # for token in tokens:
    #     print(token)

In [35]:
def decode_tokens(tokens, tokenizer_path):
    # Initialize the tokenizer
    tokenizer = Tokenizer.from_file(tokenizer_path)

    # Decode the tokens
    decoded_text = tokenizer.decode(tokens)

    return decoded_text



Siddhesh tokenizer 30k

In [36]:
tokens = tokenize_latex_file("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", "/home/patidarritesh/30k_tokenizer.json")
tokens
decode_tokens(tokens, "/home/patidarritesh/30k_tokenizer.json")

[93, 4362, 1838, 93, 4354, 124, 4313, 2630, 2897, 126, 93, 1649, 124, 12032, 2687, 2175, 1008, 20195, 1027, 15149, 11425, 7762, 1050, 22221, 126, 93, 1166, 124, 7685, 5882, 126, 93, 3635, 124, 4650, 1025, 7414, 2727, 45, 3706, 10994, 1860, 7244, 2544, 1038, 20278, 1312, 45, 4678, 3392, 20278, 1312, 45, 7892, 126, 93, 1166, 124, 2650, 9535, 5060, 1363, 126, 93, 6159, 124, 2635, 9535, 47, 1011, 1363, 65, 2949, 46, 101, 1154, 1312, 47, 1031, 126, 93, 3635, 124, 4650, 1025, 7414, 2727, 45, 3706, 10994, 1860, 7244, 2544, 1038, 20278, 1312, 45, 4678, 3392, 20278, 1312, 45, 7892, 126, 93, 3083, 1022, 8901, 126, 93, 1116, 124, 3041, 126, 1111, 6322, 2092, 1025, 11425, 7762, 1050, 22221, 1023, 14394, 8231, 45, 4167, 1039, 1015, 8888, 1025, 5942, 13377, 1008, 1015, 2626, 2092, 1054, 1015, 15149, 2032, 47, 19687, 1144, 1015, 11701, 1120, 5942, 13377, 1008, 8991, 45, 1079, 7074, 1015, 2562, 3886, 1025, 14394, 15942, 2687, 2175, 47, 29500, 1072, 45, 5994, 1025, 2687, 2175, 2778, 2510, 1008, 1015, 2

'\\importpackages{}\\graphicspath{{./images/}}\\title{Topologicalsurfacestatesinparamagneticandantiferromagneticironpnictides}\\author{AlexanderLau}\\affiliation{InstituteofTheoreticalPhysics,TechnischeUniversit\\"atDresden,01062Dresden,Germany}\\author{CarstenTimm}\\email{carsten.timm@tu-dresden.de}\\affiliation{InstituteofTheoreticalPhysics,TechnischeUniversit\\"atDresden,01062Dresden,Germany}\\date{\\today}\\begin{abstract}Theelectronicstructureofironpnictidesistopologicallynontrivial,leadingtotheappearanceofDiracconesinthebandstructurefortheantiferromagneticphase.MotivatedbytheanalogywithDiracconesingraphene,weexplorethepossibleexistenceoftopologicallyprotectedsurfacestates.Surprisingly,bandsofsurfacestatesexistevenintheparamagneticstate.Arealisticfive-orbitalmodelpredictstwosuchbands.Intheantiferromagneticphase,thesesurfacebandssurvivebutsplit.Weobtainthebulkandsurfacedispersionfromexactdiagonalizationoftwo-andfive-orbitalmodelsinastripgeometryanddiscusstheresultsbasedontopology.\

OLD 30k tokenizer

In [37]:
tokens = tokenize_latex_file("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", "/mnt/NAS/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/tokenizer.json")
tokens
decode_tokens(tokens, "/mnt/NAS/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/tokenizer.json")

[0, 64, 3853, 95, 97, 182, 64, 3840, 95, 203, 95, 18, 19, 3511, 19, 97, 203, 97, 182, 182, 182, 64, 950, 95, 19529, 2198, 1663, 277, 26606, 280, 18486, 182, 26432, 273, 14519, 245, 1449, 97, 182, 64, 436, 95, 11362, 16299, 97, 182, 64, 3203, 95, 12556, 268, 8671, 2243, 16, 3321, 14479, 1376, 9255, 64, 6, 293, 182, 40, 405, 1131, 16, 203, 20, 21, 20, 26, 22, 474, 405, 1131, 16, 8308, 97, 182, 64, 436, 95, 3003, 10869, 15052, 81, 97, 182, 64, 6055, 95, 3342, 10869, 18, 245, 708, 36, 7679, 17, 72, 405, 1131, 18, 281, 97, 182, 64, 3203, 95, 12556, 268, 8671, 2243, 16, 3321, 14479, 1376, 9255, 64, 6, 293, 182, 40, 405, 1131, 16, 203, 20, 21, 20, 26, 22, 474, 405, 1131, 16, 8308, 97, 182, 64, 2742, 95, 64, 11235, 97, 182, 64, 376, 95, 2573, 97, 182, 375, 6879, 1570, 268, 13995, 273, 14519, 245, 1449, 304, 15981, 8923, 16, 4108, 182, 411, 249, 10147, 268, 5837, 15005, 277, 249, 2520, 1570, 326, 249, 18486, 182, 5697, 18, 2056, 15691, 427, 249, 11901, 388, 5837, 15005, 277, 8610, 16, 378, 7298

'[PAD]\\importpackages{}\n\\graphicspath{ {./images/} }\n\n\n\\title{Topological surface states in paramagnetic and antiferromagnetic\niron pnictides}\n\\author{Alexander Lau}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\author{Carsten Timm}\n\\email{carsten.timm@tu-dresden.de}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\date{\\today}\n\\begin{abstract}\nThe electronic structure of iron pnictides is topologically nontrivial, leading\nto the appearance of Dirac cones in the band structure for the antiferromagnetic\nphase. Motivated by the analogy with Dirac cones in graphene, we explore the\npossible existence\nof topologically protected surface states. Surprisingly, bands of surface states\nexist even in the paramagnetic state. A realistic five-orbital model\npredicts two such bands. In the antiferromagnetic phase, these surface bands\nsurvive but split

# Nougat tokenizer 50k

In [38]:
tokenizer_path = "/home/patidarritesh/Nougat/nougat/nougat/dataset/tokenizer.json"
tokens = tokenize_latex_file("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", tokenizer_path)
print(tokens)
decode_tokens(tokens, tokenizer_path)

[0, 82, 4669, 9419, 1131, 113, 115, 221, 82, 34773, 1072, 450, 113, 243, 113, 36, 37, 17185, 37, 115, 243, 115, 221, 221, 221, 82, 18097, 113, 20666, 998, 1311, 2003, 301, 24273, 312, 21228, 221, 1380, 298, 9303, 48826, 115, 221, 82, 16004, 113, 36200, 7850, 32772, 115, 221, 82, 3121, 35095, 113, 34264, 1151, 299, 13875, 10132, 34, 4501, 28192, 37313, 82, 24, 285, 221, 58, 376, 3886, 34, 243, 38, 39, 38, 44, 40, 412, 376, 3886, 34, 6420, 115, 221, 82, 16004, 113, 11751, 15576, 8416, 99, 115, 221, 82, 390, 827, 113, 3260, 15576, 36, 549, 99, 54, 18276, 35, 90, 376, 3886, 36, 639, 115, 221, 82, 3121, 35095, 113, 34264, 1151, 299, 13875, 10132, 34, 4501, 28192, 37313, 82, 24, 285, 221, 58, 376, 3886, 34, 243, 38, 39, 38, 44, 40, 412, 376, 3886, 34, 6420, 115, 221, 82, 7913, 113, 82, 106, 19112, 115, 221, 82, 2722, 113, 28499, 115, 221, 592, 5260, 1247, 299, 5114, 298, 9303, 48826, 343, 25291, 14798, 34, 3249, 221, 862, 286, 7345, 299, 9657, 20058, 301, 286, 2144, 1247, 345, 286, 21228, 22

'\\importpackages{}\n\\graphicspath{ {./images/} }\n\n\n\\title{Topological surface states in paramagnetic and antiferromagnetic\niron pnictides}\n\\author{Alexander Lau}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\author{Carsten Timm}\n\\email{carsten.timm@tu-dresden.de}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\date{\\today}\n\\begin{abstract}\nThe electronic structure of iron pnictides is topologically nontrivial, leading\nto the appearance of Dirac cones in the band structure for the antiferromagnetic\nphase. Motivated by the analogy with Dirac cones in graphene, we explore the\npossible existence\nof topologically protected surface states. Surprisingly, bands of surface states\nexist even in the paramagnetic state. A realistic five-orbital model\npredicts two such bands. In the antiferromagnetic phase, these surface bands\nsurvive but split. We 

# New 30k BPE tokenizer  trained on 31/03/24

In [39]:
tokenizer_path = "/mnt/NAS/patidarritesh/PDF_2_LaTeX/Tokenizer/tokenizers_0.02_30000.json"
tokens = tokenize_latex_file("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", tokenizer_path)
print(tokens)
decode_tokens(tokens, tokenizer_path)

Exception: No such file or directory (os error 2)

# New 30K without whitespace trained on 31/03/24

In [4]:
tokenizer_path = "/mnt/NAS/patidarritesh/PDF_2_LaTeX/Tokenizer/tokenizer_30k_without_whiteSpace.json"
tokens = tokenize_latex_file("/mnt/NAS/patidarritesh/latex_files/1305.1770.tex", tokenizer_path)
print(tokens)
decode_tokens(tokens, tokenizer_path)

[5715, 98, 2181, 4074, 5713, 2287, 17, 17, 98, 1749, 2483, 1390, 5997, 3381, 2996, 1043, 19530, 5940, 1046, 25446, 17, 4380, 1040, 21147, 1014, 5309, 1036, 98, 1219, 2075, 19438, 1161, 1174, 1036, 98, 20429, 8379, 16459, 1035, 11682, 7842, 4903, 116, 27828, 12443, 27899, 17, 74, 1271, 27620, 1321, 1281, 5985, 1227, 1271, 27620, 22953, 98, 1219, 2270, 1024, 21909, 18183, 8065, 19153, 6484, 129, 6105, 21909, 52, 1014, 2785, 70, 21233, 2230, 1271, 4775, 52, 25504, 98, 20429, 8379, 16459, 1035, 11682, 7842, 4903, 116, 27828, 12443, 27899, 17, 74, 1271, 27620, 1321, 1281, 5985, 1227, 1271, 27620, 22953, 98, 5576, 18181, 1036, 98, 3920, 4656, 2953, 9557, 2236, 1035, 21452, 1040, 21147, 1014, 5309, 1071, 22911, 12305, 50, 3809, 2416, 1260, 1017, 13977, 1035, 8456, 28614, 1043, 1017, 3914, 2236, 1093, 1017, 25446, 17, 14703, 52, 5932, 25491, 1202, 1017, 18892, 1158, 8456, 28614, 1043, 2464, 1015, 1864, 1153, 10392, 1688, 2675, 2364, 5482, 17, 1544, 22911, 1169, 18470, 3381, 12381, 7610, 1803, 

'\\importpackages{}\n\\graphicspath{ {./images/} }\n\n\n\\title{Topological surface states in paramagnetic and antiferromagnetic\niron pnictides}\n\\author{Alexander Lau}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\author{Carsten Timm}\n\\email{carsten.timm@tu-dresden.de}\n\\affiliation{Institute of Theoretical Physics, Technische Universit\\"at\nDresden, 01062 Dresden, Germany}\n\\date{\\today}\n\\begin{abstract}\nThe electronic structure of iron pnictides is topologically nontrivial, leading\nto the appearance of Dirac cones in the band structure for the antiferromagnetic\nphase. Motivated by the analogy with Dirac cones in graphene, we explore the\npossible existence\nof topologically protected surface states. Surprisingly, bands of surface states\nexist even in the paramagnetic state. A realistic five-orbital model\npredicts two such bands. In the antiferromagnetic phase, these surface bands\nsurvive but split. We 

In [40]:
import math
import os
from typing import List, Optional, Union
import numpy as np
from PIL import Image
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer
from torchvision.transforms.functional import resize, rotate
from torchvision import transforms



class SwinEncoder(nn.Module):

    def __init__(
        self,
        input_size: List[int],
        align_long_axis: bool,
        window_size: int,
        encoder_layer: List[int],
        patch_size: int,
        embed_dim: int,
        num_heads: List[int],
        name_or_path: Union[str, bytes, os.PathLike] = None,
    ):
        super().__init__()
        self.input_size = input_size
        self.align_long_axis = align_long_axis
        self.window_size = window_size
        self.encoder_layer = encoder_layer
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.model = SwinTransformer(
            img_size=self.input_size,
            depths=self.encoder_layer,
            window_size=self.window_size,
            patch_size=self.patch_size,
            embed_dim=self.embed_dim,
            num_heads=self.num_heads,
            num_classes=0,
        )

        # weight init with swin
        if not name_or_path:
            swin_state_dict = timm.create_model(
                "swin_base_patch4_window12_384", pretrained=True
            ).state_dict()
            new_swin_state_dict = self.model.state_dict()
            for x in new_swin_state_dict:
                if x.endswith("relative_position_index") or x.endswith("attn_mask"):
                    pass
                elif (
                    x.endswith("relative_position_bias_table")
                    and self.model.layers[0].blocks[0].attn.window_size[0] != 12
                ):
                    pos_bias = swin_state_dict[x].unsqueeze(0)[0]
                    old_len = int(math.sqrt(len(pos_bias)))
                    new_len = int(2 * window_size - 1)
                    pos_bias = pos_bias.reshape(1, old_len, old_len, -1).permute(
                        0, 3, 1, 2
                    )
                    pos_bias = F.interpolate(
                        pos_bias,
                        size=(new_len, new_len),
                        mode="bicubic",
                        align_corners=False,
                    )
                    new_swin_state_dict[x] = (
                        pos_bias.permute(0, 2, 3, 1)
                        .reshape(1, new_len**2, -1)
                        .squeeze(0)
                    )
                else:
                    new_swin_state_dict[x] = swin_state_dict[x]
            self.model.load_state_dict(new_swin_state_dict)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch_size, num_channels, height, width)
        """
        x = self.model.patch_embed(x)
        x = self.model.pos_drop(x)
        x = self.model.layers(x)
        return x

    @staticmethod
    def crop_margin(img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

  
    def to_tensor(self, img: Image.Image):
        if self.training:
            
            return transforms.ToTensor()(img)
        else:
            return transforms.ToTensor()(img)
   
    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        input_size= [896,672]
        # Convert PDF to images using PyMuPDF
        doc = fitz.open(pdf_path)
        i=0
        for page_number in range(len(doc)):
            if i==1:
                break
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)

            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (input_size[0] > input_size[1] and img.width > img.height)
                or (input_size[0] < input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(input_size))
            img.thumbnail((input_size[1], input_size[0]))
            delta_width = input_size[1] - img.width
            delta_height = input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            padded_img = ImageOps.expand(img, padding)

            # resize the image (224,224)
            # padded_img= padded_img.resize((224,224))
            page_tensor = self.to_tensor(padded_img)
            # page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            # print("Page Tensor: ", page_tensor.shape)
            if self.input_tensor is None:
               self.input_tensor = page_tensor
            else:
                self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=1)
            i+=1

        # ---------------------------------------------------VERTICAL PADDING added--------------------------------------
        # target_shape = (3, self.input_size[0], self.input_size[1])
        # padding_needed = (target_shape[1] - self.input_tensor.size(1))
        # self.input_tensor = torch.nn.functional.pad(self.input_tensor, (0, 0,0, padding_needed))

        #------------------------------------------------------------------------------------------------------------------
        # img = tensor_to_image(self.input_tensor, '/home/husainmalwat/PDF_2_LaTeX/img_test_2.png')

        # print("final Input Tensor: ", )
        return self.input_tensor



In [75]:
img_processor = SwinEncoder(
            input_size=[3584,672],
            align_long_axis=False,
            window_size=7,
            encoder_layer=[2, 2, 14, 2],
            name_or_path="exp1",
            patch_size=4,
            embed_dim=128,
            num_heads=[4, 8, 16, 32],
        )

In [77]:
input_tensor = img_processor.prepare_input("/mnt/NAS/patidarritesh/swin_transformer.pdf", random_padding=True)

In [80]:
input_tensor.shape
input_tensor_grayscle = 0.299 * input_tensor[0] + 0.587 * input_tensor[1] + 0.114 * input_tensor[2]
input_tensor_grayscle.shape


torch.Size([896, 672])

In [1]:
# SWIN embedding

import os
from transformers import AutoImageProcessor, SwinModel
import torch
from datasets import load_dataset
from PIL import Image


os.environ["CUDA_VISIBLE_DEVICES"]="0"

# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]
image = "/mnt/NAS/patidarritesh/PDF_2_LaTeX/Experiments/output_1.png"

# open image file
img = Image.open(image)

image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

inputs = image_processor(img, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
img_tensor = last_hidden_states[0]

/home/patidarritesh/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificatio

In [2]:
img_tensor.shape

torch.Size([49, 768])

In [6]:
from transformers import BertModel, BertTokenizer
import fitz  # PyMuPDF

def extract_text_page_by_page(pdf_path):
    doc = fitz.open(pdf_path)
    text_pages = []
    for page_number in range(len(doc)):
        page = doc[page_number]
        text = page.get_text()
        text_pages.append(text)
    return text_pages

pdf_path = "/mnt/NAS/patidarritesh/swin_transformer.pdf"
text_pages = extract_text_page_by_page(pdf_path)
text = text_pages[0]

# pdf_path = "/mnt/NAS/patidarritesh/swin_transformer.pdf"
# text_pages = extract_text_page_by_page(pdf_path)
# # print(text)
model = BertModel.from_pretrained('mosaicml/mosaic-bert-base-seqlen-1024')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(text)

Some weights of the model checkpoint at mosaicml/mosaic-bert-base-seqlen-1024 were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'bert.encoder.layer.11.mlp.wo.weight', 'bert.encoder.layer.5.mlp.wo.weight', 'bert.encoder.layer.4.mlp.wo.bias', 'bert.encoder.layer.8.mlp.layernorm.weight', 'bert.encoder.layer.3.mlp.gated_layers.weight', 'bert.encoder.layer.11.mlp.wo.bias', 'bert.encoder.layer.11.mlp.gated_layers.weight', 'bert.encoder.layer.6.mlp.wo.bias', 'bert.encoder.layer.8.attention.self.Wqkv.bias', 'bert.encoder.layer.2.mlp.wo.weight', 'bert.encoder.layer.11.attention.self.Wqkv.bias', 'bert.encoder.layer.6.mlp.layernorm.weight', 'bert.encoder.layer.10.mlp.gated_layers.weight', 'bert.encoder.layer.4.mlp.layernorm.weight', 'bert.encoder.layer.1.mlp.wo.bias', 'bert.encoder.layer.4.attention.self.Wqkv.weight', 'bert.encoder.layer.8.mlp.wo.bias', 'bert.encoder.layer.3.mlp.wo.bias', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.9.mlp.wo.weight', 'cls.

In [7]:
len(tokens)

869

In [8]:
import torch


input_ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids_new = []
for i in range(0,len(input_ids)//512):
    input_ids_new.append(input_ids[i*512:i*512+512])
input_ids_new.append(input_ids[(i+1)*512:len(input_ids)])

embeddings = []
for i in range(len(input_ids_new)):
    input_ids_new[i] = torch.tensor(input_ids_new[i]).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids_new[i])
        embeddings.append(outputs.last_hidden_state[0])
print((embeddings[0]).shape)
# input_ids = torch.tensor(input_ids).unsqueeze(0)



torch.Size([512, 768])


In [9]:
embeddings_new = torch.concat(embeddings, dim=0)

print(embeddings_new.shape)

torch.Size([869, 768])


In [27]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [41]:
# code to combine tensors with a linear layer

import torch
import torch.nn as nn

tensor1 = embeddings_new
tensor2 = img_tensor

padding1 = 1024 - tensor1.size(0)
padding2 = 49 - tensor2.size(0) # 588 for final image size of 896*672

# Pad the tensor with zeros to make it of size 500x500
padded_tensor_1 = torch.nn.functional.pad(tensor1, (0, 0, 0, padding1))
padded_tensor_2 = torch.nn.functional.pad(tensor2, (0, 0, 0, padding2))

# Concatenate the flattened embeddings
combined_tensor = torch.cat((padded_tensor_1, padded_tensor_2), dim=0)

with torch.no_grad():   
    linear_layer = nn.Linear(768,1000)
    output = linear_layer(combined_tensor)

# output_tensor = linear_layer(combined_tensor)

In [42]:
output.shape

torch.Size([1073, 1000])

In [43]:
padded_tensor_2.shape[0]

49

In [44]:
tensor2.shape[0]

49

In [45]:
print(combined_tensor)

tensor([[-0.9732, -0.3425, -0.2130,  ...,  0.3627, -0.4804,  0.6546],
        [-1.7553, -1.0483,  0.6857,  ...,  0.2230, -0.1326,  0.1200],
        [-1.2787, -1.0483, -0.1752,  ...,  1.1283, -0.2527,  1.1791],
        ...,
        [-0.3335,  0.3159,  0.2549,  ..., -0.0097,  0.3009,  0.4155],
        [-0.1315,  0.3890,  0.3493,  ...,  0.1082,  0.2266,  0.2570],
        [ 0.5022, -0.0506,  0.3930,  ...,  0.3663, -0.0213,  0.6247]])


In [46]:
print(output)

tensor([[-0.0476, -0.6364, -0.3562,  ..., -0.6168, -0.3195, -0.1962],
        [-0.2231,  0.3488, -0.1409,  ..., -0.7402, -0.1919, -0.3527],
        [ 0.2109,  0.0545, -0.4859,  ..., -0.4634,  0.2458,  0.1142],
        ...,
        [-0.1514,  0.0286, -0.3471,  ...,  0.0236,  0.1454,  0.0763],
        [-0.1960, -0.0190, -0.3103,  ..., -0.0895,  0.1193, -0.0025],
        [-0.1134,  0.3029, -0.4336,  ..., -0.3468,  0.1653,  0.1559]])


In [1]:
import torch
import torch.nn as nn
tensor1 = torch.randn(5, 3)
tensor2 = torch.randn(5, 3) 
combined_tensor = torch.cat((tensor1, tensor2), dim=1) 
with torch.no_grad(): #do not need gradients in this case
    linear_layer = nn.Linear(in_features=(tensor1.shape[1]+tensor2.shape[1]), out_features=tensor1.shape[1]) #output shape is original shape 
    output_tensor = linear_layer(combined_tensor)

In [4]:
print(combined_tensor.shape)
print(output_tensor.shape)

torch.Size([5, 6])
torch.Size([5, 3])


In [3]:
import logging
import math
import os
from typing import List, Optional, Union
from collections import defaultdict
from pathlib import Path

import numpy as np
from PIL import Image
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer
from torchvision.transforms.functional import resize, rotate
from transformers import (
    BertModel,
    BertTokenizer,
    BertConfig,
    PreTrainedTokenizerFast,
    StoppingCriteria,
    StoppingCriteriaList,
    MBartConfig,
    MBartForCausalLM,
)
from transformers.file_utils import ModelOutput
from transformers.modeling_utils import PretrainedConfig, PreTrainedModel
from torchvision import transforms
import fitz # PyMuPDF

class modifiedBERTEncoder(nn.Module):
    """
    Encoder based on BERT (Bidirectional Encoder Representations from Transformers)

    Args:
        encoder_layer:
            Number of layers of the BERT encoder.
        max_position_embeddings:
            The maximum sequence length to be trained.
        hidden_dimension:
            The dimensionality of the hidden layers.
        name_or_path:
            Name of a pretrained model name either registered in huggingface.co or saved locally.
            Otherwise, `bert-base-multilingual-cased` will be set.
    """

    def __init__(
        self,
        # encoder_layer: int,
        max_position_embeddings: int,
        hidden_size: int,
        num_attention_heads: int,
        txt_enc_name_or_path: Union[str, bytes, Path] = None,
    ):
        super().__init__()
        # self.encoder_layer = encoder_layer
        self.max_position_embeddings = max_position_embeddings
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        

        self.model = BertModel(
            config = BertConfig(
                hidden_size=1024,
                max_position_embeddings=1024,
                num_attention_heads=16
            )
        )
        
        # Initialize BERT model
        if not txt_enc_name_or_path:
            txt_enc_name_or_path = "bert-base-uncased"
        self.model = BertModel.from_pretrained(txt_enc_name_or_path)


In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [4]:
model = modifiedBERTEncoder(
            max_position_embeddings=1024,
            hidden_size=1024,
            num_attention_heads=16,
            txt_enc_name_or_path="bert-base-uncased"
            )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import pickle
import torch

# Create sample tensors
tensor1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor2 = torch.tensor([[7, 8, 9], [10, 11, 12]])

# Create a dictionary to store the tensors
tensors_dict = {'tensor1': tensor1, 'tensor2': tensor2}

# Specify the file path where you want to store the tensors
file_path = "tensors_data.pickle"

# Open a file in binary write mode
with open(file_path, "wb") as file:
    # Use pickle to dump the dictionary containing tensors into the file
    pickle.dump(tensors_dict, file)

print("Tensors have been stored in", file_path)


In [6]:
import os
print(len(os.listdir('/mnt/NAS/patidarritesh/grounding_text_PDF_2_LaTeX/pdf_2_tex/dataset/img_tensor')), len(os.listdir('/mnt/NAS/patidarritesh/grounding_text_PDF_2_LaTeX/pdf_2_tex/dataset/latex_tensor')), len(os.listdir('/mnt/NAS/patidarritesh/grounding_text_PDF_2_LaTeX/pdf_2_tex/dataset/text_tensor')))

1168 1168 1168


In [4]:
pth = "/mnt/NAS/patidarritesh/5_page_pdfs/1110.5406.pdf"

print(pth.split("/")[-1])

1110.5406.pdf


In [11]:
import pickle


# open  a pickle file from a path
file_path = "/mnt/NAS/patidarritesh/grounding_text_PDF_2_LaTeX/pdf_2_tex/dataset/img_tensor/0704.3345.pdf.pkl"
with open(file_path, "rb") as file:
    # Load the dictionary containing tensors from the file
    tensors_dict = pickle.load(file)

print(type(tensors_dict))
# with open(img_file, "r") as f:
#     self.input_tensor = f.read()
#     self.input_tensor = torch.tensor(self.input_tensor)

<class 'torch.Tensor'>
